In [2]:
from ultralytics import YOLO
model = YOLO('best.pt')

In [ ]:
model.predict(source=0, show=True)


1/1: 0... Success  (inf frames of shape 640x480 at 30.00 FPS)


WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

0: 480x640 (no detections), 220.5ms
0: 480x640 (no detections), 616.3ms
0: 480x640 (no detections), 571.4ms
0: 480x640 (no detections), 232.5ms
0: 480x640 (no detections), 209.8ms
0: 480x640 (no detections), 133.1ms
0: 480x640 1 open_manhole, 145.9ms
0: 480x640 (no detections), 155.6ms
0: 480x640 (no detections), 132.6ms
0: 480x640 1 open_manhole, 145.0ms
0: 480x640 1 open_manhole, 143.4ms
0:

KeyboardInterrupt: 

: 

In [ ]:
# Notebook cell: webcam detection (inline display)
import time, cv2, numpy as np
from ultralytics import YOLO
from IPython.display import display, clear_output
from PIL import Image

model = YOLO("best.pt")
model.to("cuda")  # or "cpu"

cap = cv2.VideoCapture(0)
CONF_THRESH = 0.35

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Optionally resize
        # frame = cv2.resize(frame, (640,360))

        results = model(frame, imgsz=640, conf=CONF_THRESH, verbose=False)
        r = results[0]
        if hasattr(r, "boxes") and len(r.boxes) > 0:
            boxes = r.boxes.xyxy.cpu().numpy()
            confs = r.boxes.conf.cpu().numpy()
            cls_ids = r.boxes.cls.cpu().numpy()
            for (x1, y1, x2, y2), conf, cls in zip(boxes.astype(int), confs, cls_ids.astype(int)):
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0,168,255), 2)
                cv2.putText(frame, f"{conf:.2f}", (x1, y1-6), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)

        # Convert BGR->RGB and display
        display_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(display_frame)
        clear_output(wait=True)
        display(img)

        # small sleep to yield to notebook
        time.sleep(0.05)
except KeyboardInterrupt:
    pass
finally:
    cap.release()
    clear_output()
    print("Stopped")